In [11]:
import torch
import torch.nn as nn
from torchvision import transforms, datasets
from PIL import Image

# Importar tu modelo definido en model.py
from model import ElectronicComponentCNN

# === CONFIGURACIÓN ===
data_dir = "./train_data"   # carpeta con las clases
weights_path = "./checkpoint/model_final.pth"  # ruta a tu archivo .pth
image_path = "./test_data/resistor/ScreenShot2022-10-24at10.56.33AM_64bd43af-8820-41bf-9469-cf994e246838_1024x1024.png"

# === OBTENER CLASES DEL DATASET ===
dataset = datasets.ImageFolder(root=data_dir)
class_names = dataset.classes
num_classes = len(class_names)

# === CARGAR MODELO ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ElectronicComponentCNN(num_classes=num_classes)
model.load_state_dict(torch.load(weights_path, map_location=device))
model.to(device)
model.eval()

# === TRANSFORMACIONES DE LA IMAGEN ===
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),   # convertir a escala de grises
    transforms.Resize((256, 256)),                 # redimensionar a 256x256
    transforms.ToTensor(),                         # convertir a tensor
    transforms.Normalize(mean=[0.5], std=[0.5])    # normalizar [-1,1]
])

# === CARGAR Y PREPROCESAR IMAGEN ===
img = Image.open(image_path).convert("L")  # asegurar escala de grises
img_tensor = transform(img).unsqueeze(0).to(device)  # añadir batch dimension

# === INFERENCIA ===
with torch.no_grad():
    output = model(img_tensor)
    predicted_class = torch.argmax(output, dim=1).item()

print(f"Predicción: Clase {predicted_class} -> {class_names[predicted_class]}")



Predicción: Clase 0 -> push


In [13]:
import torch
from torchvision import transforms, datasets
from PIL import Image
import glob
import os
import pandas as pd

# Importar tu modelo definido en model.py
from model import ElectronicComponentCNN

# === CONFIGURACIÓN ===
data_dir = "./train_data"   # carpeta con las clases
weights_path = "./checkpoint/model_final.pth"  # ruta a tu archivo .pth
test_folder = "./test_data/resistor"           # carpeta con imágenes a inferir

# === OBTENER CLASES DEL DATASET ===
dataset = datasets.ImageFolder(root=data_dir)
class_names = dataset.classes
num_classes = len(class_names)

# === CARGAR MODELO ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ElectronicComponentCNN(num_classes=num_classes)
model.load_state_dict(torch.load(weights_path, map_location=device))
model.to(device)
model.eval()

# === TRANSFORMACIONES DE LA IMAGEN ===
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),   # convertir a escala de grises
    transforms.Resize((256, 256)),                 # redimensionar a 256x256
    transforms.ToTensor(),                         # convertir a tensor
    transforms.Normalize(mean=[0.5], std=[0.5])    # normalizar [-1,1]
])

# === INFERENCIA SOBRE TODAS LAS IMÁGENES DE LA CARPETA ===
image_paths = glob.glob(os.path.join(test_folder, "*.*"))  # todas las imágenes

results = []
for img_path in image_paths:
    img = Image.open(img_path).convert("L")
    img_tensor = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(img_tensor)
        predicted_class = torch.argmax(output, dim=1).item()

    results.append({
        "Archivo": os.path.basename(img_path),
        "Clase (índice)": predicted_class,
        "Etiqueta": class_names[predicted_class]
    })

# === MOSTRAR RESULTADOS EN TABLA ===
df = pd.DataFrame(results)
print(df.to_string(index=False))


                                                                                                          Archivo  Clase (índice) Etiqueta
                                                                                 istockphoto-89444107-612x612.jpg               1 resistor
                                                                                         resistor-rvAElYD-600.jpg               1 resistor
108870118-resistor-passive-two-terminal-electrical-component-for-electrical-resistance-to-reduce-current-flow.jpg               1 resistor
                                                                                                  images (2).jpeg               1 resistor
                              ScreenShot2022-10-24at10.56.33AM_64bd43af-8820-41bf-9469-cf994e246838_1024x1024.png               0     push
                                                                                                  images (1).jpeg               1 resistor
